In [0]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./drive


Bism  
# Imports

In [0]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as sklm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, Adam

# Implementation 10  (PHONE LEVEL)
## Using Cambridge as Training set
## L2 as val+test set
## AUTOENCODER PER PHONE

In [0]:
phones = [file[:-4] for file in os.listdir('./drive/My Drive/Thesis Work/Cambridge Phones/')]
len(phones)

42

In [0]:
phone = "L"

### Training

#### X_TRAIN, X_TEST & Y_TEST

In [0]:
X = np.load('./drive/My Drive/Thesis Work/Cambridge Phones/'+phone+'.npy')
np.random.shuffle(X)
X.shape

(1093, 13, 50, 3)

In [0]:
X_train = X[:int(np.ceil(X.shape[0]*0.9))]
X_test = X[int(np.ceil(X.shape[0]*0.9)):]
y_test = np.ones(X_test.shape[0])
X_train.shape[0]+X_test.shape[0]

1093

In [0]:
y_test.shape

(109,)

In [0]:
for p in phones:
  if p != phone:
    notPhone = np.load('./drive/My Drive/Thesis Work/Cambridge Phones/'+p+'.npy')
    np.random.shuffle(notPhone)
    X_test = np.vstack((X_test,notPhone[:int(np.ceil(y_test.shape[0]/41))]))

In [0]:
y_test = np.hstack((y_test,np.zeros(int(np.ceil(y_test.shape[0]/41)*41))))
y_test.shape

(232,)

In [0]:
X_test.shape

(232, 13, 50, 3)

In [0]:
X_train.shape

(984, 13, 50, 3)

#### MODEL

In [0]:
#commented to avoid accedental runs
'''for phone in set(train_phones):
  np.save("./drive/My Drive/Thesis Work/Implementation6/phone_means/mean_"+phone+".npy",\
          np.max(np.abs(x_trainALL[train_phones == phone]),(0,1,2)))''';

In [0]:
train_max = np.load('./drive/My Drive/Thesis Work/L2 Phones/train_phones_means/mean_'+phone+".npy") #???
X_train = X_train / train_max
print(phone)
print(X_train.shape)

L
(984, 13, 50, 3)


In [0]:
print(np.max(X_train))
print(np.min(X_train))

0.7820627989513965
-1.240481751809753


In [0]:
np.nan in X_train

False

In [0]:
'''noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_train_noisy = np.clip(x_train_noisy, -1, 1.)''';

In [0]:
input_img = Input(shape=(13, 50, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(64, (2, 3), activation='tanh', padding='valid')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='tanh', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (3, 12, 32)

x = BatchNormalization()(encoded)
x = Conv2D(128, (3, 3), activation='tanh', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='tanh', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
decoded = Conv2DTranspose(3, (2, 3), activation='tanh', padding='valid')(x)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer= keras.optimizers.Adam(), loss='mean_squared_error');

In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 13, 50, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 48, 64)        1216      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 24, 64)         0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 6, 24, 64)         256       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 24, 128)        73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 12, 128)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 3, 12, 128)        512       
__________

In [0]:
callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)]
autoencoder.fit(X_train, X_train, batch_size=32, epochs=5, callbacks=callbacklist)

Epoch 1/5
984/984 [==============================] - 1s 954us/step - loss: 3.4366e-04
Epoch 2/5
984/984 [==============================] - 1s 949us/step - loss: 3.1935e-04
Epoch 3/5
984/984 [==============================] - 1s 952us/step - loss: 3.0562e-04
Epoch 4/5
984/984 [==============================] - 1s 942us/step - loss: 3.0260e-04
Epoch 5/5
984/984 [==============================] - 1s 955us/step - loss: 3.3670e-04

Epoch 00005: loss improved from inf to 0.00034, saving model to ./jawadmodel.h5


In [0]:
modelLoss = _.history['loss'][-1]
modelLoss

0.0003367000246681727

In [0]:
phone

'L'

In [0]:
!cp ./jawadmodel.h5 "./drive/My Drive/Thesis Work/PhoneLevel_AutoEncoder_TrainCamb_TestCamb/"$phone".h5"

### TESTING

In [0]:
folder = '"./drive/My Drive/Thesis Work/PhoneLevel_AutoEncoder_MFCC_TrainCamb_TestCamb/"'

In [0]:
!ls $folder

AA.h5  B.h5  L.h5  R.h5


In [0]:
#MAX normalization
train_max = np.load('./drive/My Drive/Thesis Work/L2 Phones/train_phones_means/mean_'+phone+".npy")
X_test = X_test / train_max
print(phone)
print(X_test.shape)
print(y_test.shape)

L
(232, 13, 50, 3)
(232,)


In [0]:
'''noise_factor = 0.5
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
x_test_noisy = np.clip(x_test_noisy, -1, 1)''';

In [0]:
autoencoder = load_model(folder[1:-1]+phone+'.h5') #load previous one without noise

In [0]:
autoencoder.evaluate(X_test[y_test == 1],X_test[y_test == 1],batch_size=1)

109/109 [==============================] - 1s 8ms/step


0.00034479620474510306

In [0]:
autoencoder.evaluate(X_test[y_test == 0],X_test[y_test == 0],batch_size=1)

123/123 [==============================] - 0s 3ms/step


0.0008833857675081088

In [0]:
#Need to shuffle X_test

In [0]:
shuffledIndexes = np.random.permutation(range(X_test.shape[0]))
X_test = X_test[shuffledIndexes]
y_test = y_test[shuffledIndexes]

In [0]:
X_test.shape

(232, 13, 50, 3)

In [0]:
validationSetLength = 70 #initial 200 from test as validation set
accs = []
f1mis = []

for threshold in np.arange(0.0001,0.001,0.00001):
  y_pred = [1 if autoencoder.evaluate(X_test[i:i+1],X_test[i:i+1],verbose=0)<=threshold\
   else 0 for i in range(validationSetLength)]
  
  accs.append(sklm.accuracy_score(y_test[:validationSetLength],y_pred))
  f1mis.append(sklm.precision_recall_fscore_support(y_test[:validationSetLength],y_pred)[2][0])

print("Max Achievable Accuracy (Overall): ", max(accs))
print("Max Achievable Accuracy (Overall) on Threshold: ", np.argmax(accs))

print("Max Achievable F-1 score Threshold: ", np.argmax(f1mis))
print("Max Achievable Accuracy On This Threshold: ", accs[np.argmax(f1mis)])
print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Max Achievable Accuracy (Overall):  0.7428571428571429
Max Achievable Accuracy (Overall) on Threshold:  36
Max Achievable F-1 score Threshold:  19
Max Achievable Accuracy On This Threshold:  0.6571428571428571
Max Achievable F-1 score for Mispronunciations:  0.7209302325581395


In [0]:
0.0001+0.00001*19

0.00029

In [0]:
validationSetLength = 70 #initial some from test as validation set
threshold = 0.00029

y_pred = [1 if autoencoder.evaluate(X_test[i:i+1],X_test[i:i+1],verbose=0)<=threshold\
 else 0 for i in range(validationSetLength,X_test.shape[0])] 

In [0]:
print("Phone: ",phone)
print(classification_report(y_test[validationSetLength:], y_pred, target_names=['Mispronounced','Correct']))
print(confusion_matrix(y_test[validationSetLength:], y_pred, labels=range(2)))

print(sklm.roc_curve(y_test[validationSetLength:],y_pred))
print('FAR = ', sklm.roc_curve(y_test[validationSetLength:],y_pred)[0][1]*100,'%')
print('FRR = ', (1 - sklm.roc_curve(y_test[validationSetLength:],y_pred)[1][1])*100,'%')
print("AUC_SCORE = ", sklm.roc_auc_score(y_test[validationSetLength:],y_pred)*100,'%')

Phone:  L
               precision    recall  f1-score   support

Mispronounced       0.65      0.92      0.76        90
      Correct       0.79      0.38      0.51        72

    micro avg       0.68      0.68      0.68       162
    macro avg       0.72      0.65      0.64       162
 weighted avg       0.71      0.68      0.65       162

[[83  7]
 [45 27]]
(array([0.        , 0.07777778, 1.        ]), array([0.   , 0.375, 1.   ]), array([2, 1, 0]))
FAR =  7.777777777777778 %
FRR =  62.5 %
AUC_SCORE =  64.86111111111111 %


*CANNOT COMPARE RESULTS:*  
The Interspeech one has a **different dataset** and shows results on phone level (as every phone is a separate class there)

The l2-arctic one shows no results

I am the first one to show results on this dataset